In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
tornado911 = pd.read_csv('../data/911_Phone_Calls_Tornado_030320.csv')
tornado_cad = pd.read_csv('../data/Computer Aided Dispatch Data_Tornado Incidents 030320.csv', skiprows = 3)
derecho911 = pd.read_csv('../data/911_Phone Calls_Derecho_050320_050420.csv')
derecho_cad = pd.read_csv('../data/Computer Aided Dispatch Data_Derecho Incidents 050320-050420.csv', skiprows = 3)
damage_points = gpd.read_file('../data/damage_points.geojson')
path_polygons = gpd.read_file('../data/path_polygons.geojson')
tornado_paths = gpd.read_file('../data/tornado_paths.geojson')

In [3]:
#chekcing out the tornado df
tornado911.head(10)

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address
0,3/3/2020 0:34,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector
1,3/3/2020 0:36,911 Calls,36.159634,-86.841904,4507D KENTUCKY AV - SE
2,3/3/2020 0:36,911 Calls,36.264153,-86.708862,329 THELMA ST - S Sector
3,3/3/2020 0:36,911 Calls,NaN,NaN,526 FOUNDRY DR
4,3/3/2020 0:36,911 Calls,36.233467,-86.721077,5007B RUSKIN AVE - N
5,3/3/2020 0:37,911 Calls,36.149032,-86.795747,WIRELESS CALLER
6,3/3/2020 0:37,911 Calls,36.175811,-86.895654,WIRELESS CALLER
7,3/3/2020 0:37,911 Calls,36.180500,-86.883681,WIRELESS CALLER
8,3/3/2020 0:38,911 Calls,36.070980,-86.939964,201 FORREST VALLEY DR - SECTOR S
9,3/3/2020 0:38,911 Calls,36.163237,-86.779654,415 CHURCH ST


In [4]:
tornado911.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Seizure DateTime    475 non-null    object 
 1   CallTypeId          475 non-null    object 
 2   ALI Latitude        450 non-null    float64
 3   ALI Longitude       450 non-null    float64
 4   Cell Tower Address  468 non-null    object 
dtypes: float64(2), object(3)
memory usage: 18.7+ KB


In [5]:
tornado911.shape

(475, 5)

In [6]:
type(tornado911)

pandas.core.frame.DataFrame

In [7]:
###Dropping the rows from ALI Latitude that contain null values.  I was going to do another piece of code that also dropped null
###values for ALI Longitude but it appears, that by dropping the null lat rows; those same rows contained null long.
tornado911 = tornado911.loc[tornado911['ALI Latitude'].notnull()]

In [8]:
#CREATING A GEOMETRY COLUMN AND DATA FOR TORNADO911 DF
tornado911['geometry'] = tornado911.apply(lambda x: Point((float(x['ALI Longitude']),
                                                           float(x['ALI Latitude']))),
                                          axis=1)
tornado911.head(2)

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address,geometry
0,3/3/2020 0:34,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector,POINT (-86.665779 36.072708)
1,3/3/2020 0:36,911 Calls,36.159634,-86.841904,4507D KENTUCKY AV - SE,POINT (-86.841904 36.159634)


In [9]:
tornado911_geo = gpd.GeoDataFrame(tornado911, crs=('EPSG:4326'),geometry = tornado911['geometry'])
tornado911_geo.head()

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address,geometry
0,3/3/2020 0:34,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector,POINT (-86.66578 36.07271)
1,3/3/2020 0:36,911 Calls,36.159634,-86.841904,4507D KENTUCKY AV - SE,POINT (-86.84190 36.15963)
2,3/3/2020 0:36,911 Calls,36.264153,-86.708862,329 THELMA ST - S Sector,POINT (-86.70886 36.26415)
4,3/3/2020 0:36,911 Calls,36.233467,-86.721077,5007B RUSKIN AVE - N,POINT (-86.72108 36.23347)
5,3/3/2020 0:37,911 Calls,36.149032,-86.795747,WIRELESS CALLER,POINT (-86.79575 36.14903)


In [10]:
#checking out the tornado_cad df
tornado_cad

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1
0,3/3/2020 12:36:34 AM,6205 COCKRILL BEND CIR,36.182002,-86.894052,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD
1,3/3/2020 12:37:24 AM,7337 COCKRILL BEND BLVD,36.175666,-86.894978,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD
2,3/3/2020 12:38:17 AM,110 TUNE AIRPORT DR,36.179299,-86.883727,STRUCTURE FIRE COMMERCIAL INDUSTRIAL
3,3/3/2020 12:39:40 AM,555 MARRIOTT DR,36.151108,-86.689921,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD
4,3/3/2020 12:39:42 AM,1414 COUNTY HOSPITAL RD,36.177926,-86.845934,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD
...,...,...,...,...,...
255,3/3/2020 4:53:43 AM,1616 23RD AV N,36.175739,-86.813173,WIRES DOWN WITH NO SMOKE OR ARCING POWER DISCO...
256,3/3/2020 4:56:56 AM,5610 OLD HICKORY BLVD,36.180487,-86.601466,LARGE BUILDING INVESTIGATION
257,3/3/2020 4:57:06 AM,2903 MORNINGSIDE DR,36.223554,-86.757949,CHEST PAIN - ABNORMAL BREATHING
258,3/3/2020 4:57:14 AM,1730 KELLOW ST,36.189445,-86.813466,ABNORMAL BREATHING OTHER LUNG PROBLEMS


In [11]:
tornado_cad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   IncidentDate              260 non-null    object 
 1   Location                  260 non-null    object 
 2   Latitude1                 260 non-null    float64
 3   Longitude1                260 non-null    float64
 4   IncidentTypeDescription1  260 non-null    object 
dtypes: float64(2), object(3)
memory usage: 10.3+ KB


In [12]:
###CREATING A GEOMETRY COLUMN AND DATA FOR TORNADO_CAD DF
tornado_cad['geometry'] = tornado_cad.apply(lambda x: Point((float(x.Longitude1),
                                                             float(x.Latitude1))),
                                           axis=1)
tornado_cad.head(2)

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1,geometry
0,3/3/2020 12:36:34 AM,6205 COCKRILL BEND CIR,36.182002,-86.894052,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89405212 36.18200191)
1,3/3/2020 12:37:24 AM,7337 COCKRILL BEND BLVD,36.175666,-86.894978,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89497756999999 36.175666)


In [13]:
tornado_cad_geo = gpd.GeoDataFrame(tornado_cad, crs=('EPSG:4326'),geometry = tornado_cad['geometry'])
tornado_cad_geo.head(2)

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1,geometry
0,3/3/2020 12:36:34 AM,6205 COCKRILL BEND CIR,36.182002,-86.894052,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89405 36.18200)
1,3/3/2020 12:37:24 AM,7337 COCKRILL BEND BLVD,36.175666,-86.894978,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89498 36.17567)


In [14]:
#checking out derecho df
derecho911

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address
0,5/3/2020 16:30,911 Calls,36.247802,-86.716847,WIRELESS CALLER
1,5/3/2020 16:30,911 Calls,36.143108,-86.800621,1161 21ST AV S - SE
2,5/3/2020 16:30,911 Calls,36.100516,-87.056329,8013 C MCCRORY LN - N
3,5/3/2020 16:30,911 Calls,36.130021,-86.927819,5758 RIVER RD - SW
4,5/3/2020 16:31,911 Calls,36.142380,-86.881882,3744B ANNEX AVE - SE
...,...,...,...,...,...
1544,5/4/2020 0:18,911 Calls,36.131694,-86.770535,1823-C 12TH AV - SECTOR SE S
1545,5/4/2020 0:22,911 Calls,36.042237,-86.743455,5244-C EDMONSON PIKE - SECTOR SE
1546,5/4/2020 0:25,911 Calls,36.077664,-86.655607,2747-B MURFREESBORO PKE - SECTOR NW
1547,5/4/2020 0:29,911 Calls,36.024706,-86.743562,302 SUMMIT VIEW DR - E


In [15]:
derecho911.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1549 entries, 0 to 1548
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Seizure DateTime    1549 non-null   object 
 1   CallTypeId          1549 non-null   object 
 2   ALI Latitude        1483 non-null   float64
 3   ALI Longitude       1483 non-null   float64
 4   Cell Tower Address  1525 non-null   object 
dtypes: float64(2), object(3)
memory usage: 60.6+ KB


In [16]:
###Dropping the rows from ALI Latitude that contain null values.  I was going to do another piece of code that also dropped null
###values for ALI Longitude but it appears, that by dropping the null lat rows; those same rows contained null long.
derecho911 = derecho911.loc[derecho911['ALI Latitude'].notnull()]

In [17]:
derecho911.shape

(1483, 5)

In [18]:
###CREATING A GEOMETRY COLUMN AND DATA FOR DERECHO911 DF
derecho911['geometry'] = derecho911.apply(lambda x: Point((float(x['ALI Longitude']),
                                                           float(x['ALI Latitude']))),
                                         axis=1)
derecho911.head(2)

C:\Users\hbn2b\AppData\Local\Temp/ipykernel_16456/408533418.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derecho911['geometry'] = derecho911.apply(lambda x: Point((float(x['ALI Longitude']),


,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address,geometry
0,5/3/2020 16:30,911 Calls,36.247802,-86.716847,WIRELESS CALLER,POINT (-86.716847 36.247802)
1,5/3/2020 16:30,911 Calls,36.143108,-86.800621,1161 21ST AV S - SE,POINT (-86.80062100000001 36.143108)


In [19]:
derecho911_geo = gpd.GeoDataFrame(derecho911, crs=('EPSG:4326'),geometry = derecho911['geometry'])
derecho911_geo.head(2)

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address,geometry
0,5/3/2020 16:30,911 Calls,36.247802,-86.716847,WIRELESS CALLER,POINT (-86.71685 36.24780)
1,5/3/2020 16:30,911 Calls,36.143108,-86.800621,1161 21ST AV S - SE,POINT (-86.80062 36.14311)


In [20]:
#checking out derecho_cad df
derecho_cad

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1
0,5/3/2020 4:31:07 PM,901 KENWICK CT W,36.046877,-86.962526,ELECTRICAL HAZARD
1,5/3/2020 4:32:18 PM,8036 ARBOR DR,36.082491,-86.940680,ELECTRICAL HAZARD
2,5/3/2020 4:32:23 PM,119 DUE WEST AV E,36.248029,-86.716875,NOT ALERT COPD
3,5/3/2020 4:33:04 PM,630 GALLATIN PKE S,36.256491,-86.714817,ABDOMINAL PAIN
4,5/3/2020 4:33:52 PM,615 W HILLWOOD DR,36.129690,-86.879834,ELECTRICAL HAZARD
...,...,...,...,...,...
622,5/4/2020 12:21:00 AM,625 BENTON AV,36.131384,-86.770628,HIGH RISE NATURAL GAS ODOR
623,5/4/2020 12:21:32 AM,518 BRENTLAWN DR,36.064362,-86.761199,UNKNOWN STATUS
624,5/4/2020 12:22:47 AM,517 ALBANY DR,36.212754,-86.600431,ELECTRICAL HAZARD
625,5/4/2020 12:27:52 AM,639 LAFAYETTE ST,36.153769,-86.777520,SICK PERSON: NO PRIORITY SYMPTOMS


In [21]:
derecho_cad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   IncidentDate              627 non-null    object 
 1   Location                  627 non-null    object 
 2   Latitude1                 627 non-null    float64
 3   Longitude1                627 non-null    float64
 4   IncidentTypeDescription1  627 non-null    object 
dtypes: float64(2), object(3)
memory usage: 24.6+ KB


In [22]:
#CREATING A GEOMETRY COLUMN AND DATA FOR DERECHO_CAD DF
derecho_cad['geometry'] = derecho_cad.apply(lambda x: Point((float(x.Longitude1),
                                                             float(x.Latitude1))),
                                            axis=1)
derecho_cad.head(2)

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1,geometry
0,5/3/2020 4:31:07 PM,901 KENWICK CT W,36.046877,-86.962526,ELECTRICAL HAZARD,POINT (-86.96252641 36.04687715)
1,5/3/2020 4:32:18 PM,8036 ARBOR DR,36.082491,-86.940680,ELECTRICAL HAZARD,POINT (-86.94067952 36.08249081)


In [23]:
derecho_cad_geo = gpd.GeoDataFrame(derecho_cad, crs=('EPSG:4326'),geometry = derecho_cad['geometry'])
derecho_cad_geo.head(2)

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1,geometry
0,5/3/2020 4:31:07 PM,901 KENWICK CT W,36.046877,-86.962526,ELECTRICAL HAZARD,POINT (-86.96253 36.04688)
1,5/3/2020 4:32:18 PM,8036 ARBOR DR,36.082491,-86.940680,ELECTRICAL HAZARD,POINT (-86.94068 36.08249)


In [24]:
#Checking to see if all the calls in CallTypeId column are 911 calls or if there are any other types; for instance calls to
# 862-8600.   ALL CALLS ARE 911 CALLS.
print(tornado911.CallTypeId.count())
derecho911.CallTypeId.count()

450


1483

In [25]:
#looking at value counts to see how many different entries there were and what the most common are
tornado_cad.IncidentTypeDescription1.value_counts()

GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD    27
OUTSIDE ODOR                                   26
WIRES DOWN WITH NO SMOKE OR ARCING             17
SMALL BUILDING INVESTIGATION                   15
ELECTRICAL HAZARD WITH OR NEAR WATER           12
                                               ..
WIRES DOWN WITH SMOKE OR ARCING                 1
BACKUP STATION                                  1
ENTRAPMENT UNKNOWN SITUATION                    1
TRANSFORMER OUTSIDE  WIRE OR POLE               1
UNKNOWN STATUS:                                 1
Name: IncidentTypeDescription1, Length: 90, dtype: int64

In [26]:
#looking at value counts to see how many different entries and what the most prevalent are
derecho_cad.IncidentTypeDescription1.value_counts()

ELECTRICAL HAZARD                              370
GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD     23
MVA INJURIES:                                    8
SMALL OUTSIDE FIRE                               6
UNKNOWN STATUS:                                  6
                                              ... 
OTHER PAIN                                       1
DIZZINESS / VERTIGO                              1
OVERDOSE - UNKNOWN STATUS                        1
SMOKE DETECTOR ALARM APARTMENT                   1
SICK PERSON: NO PRIORITY SYMPTOMS                1
Name: IncidentTypeDescription1, Length: 122, dtype: int64

In [27]:
#checking out damage_points df
damage_points

,objectid,stormdate,surveydate,event_id,damage,damage_txt,dod_txt,efscale,damage_dir,windspeed,...,device_id,qc,dod,surveytype,globalid,edit_user,edit_time,comments,path_guid,geometry
0,2047717,1583218620000,1583335680000,Mount Juliet,2,One- or Two-Family Residences (FR12),All walls collapsed,EF3,N/A,155,...,None,Y,9,None,{FE82BBAC-ABF8-4562-81F6-A527829FBFFE},None,None,none,None,POINT (-86.51628 36.18344)
1,2047909,1583221860000,1583437860000,Cookeville,27,Trees: Hardwood (TH),Trees uprooted,EF0,N/A,80,...,None,Y,3,None,{2417B433-0C00-4D6C-A059-45E0CFDA2AF6},None,None,none,None,POINT (-85.60552 36.17308)
2,2075630,1583221980000,1586759820000,Cookeville,2,One- or Two-Family Residences (FR12),"Loss of roof covering material (<20%), gutters...",EF0,N/A,80,...,None,Y,2,None,{10AC67A3-D75D-4E6E-82B8-DD307E183FC0},None,None,None,None,POINT (-85.57574 36.17740)
3,2060941,1583217780000,1585242540000,,1,Small Barns or Farm Outbuildings (SBO),Total destruction of building,EF1,N/A,95,...,None,Y,8,None,{A0744CB4-5993-4056-A13C-CF75182F5147},None,None,None,None,POINT (-86.76228 36.17340)
4,2060984,1583217780000,1585256400000,,7,Masonry Apartment or Motel Building (MAM),Loss of roof covering (<20%),EF0,N/A,80,...,None,Y,2,None,{DA83D428-084A-4ACA-A39E-DEE84AA28463},None,None,None,None,POINT (-86.75692 36.17568)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,2114432,1583219640000,1588470780000,,3,Manufactured Home - Single Wide (MHSW),Loss of shingles or partial uplift of one-piec...,EF0,N/A,75,...,None,Y,2,None,{44F412AC-5781-48C8-96BA-C55B7988DD17},None,None,None,None,POINT (-86.22961 36.19636)
3963,2114433,1583219700000,1588472700000,,2,One- or Two-Family Residences (FR12),"Loss of roof covering material (<20%), gutters...",EF0,N/A,85,...,None,Y,2,None,{874F6609-57F6-46B6-ACE9-4B699685233D},None,None,None,None,POINT (-86.20841 36.19219)
3964,2114434,1583220360000,1588482840000,,1,Small Barns or Farm Outbuildings (SBO),Total destruction of building,EF1,N/A,95,...,None,Y,8,None,{F21CA6EF-0A9C-4653-9D2A-87ECC197B98F},None,None,None,None,POINT (-86.02540 36.17764)
3965,2320477,1583217120000,1618702140000,Nashville,29,Other (O),Other damage,EF0,N/A,70,...,None,Y,1,None,{5071191E-4409-4F75-BD83-BE52ED30723C},None,None,Boathouse and dock blown northward and heavily...,None,POINT (-86.94269 36.17417)


In [28]:
#checking out path_polygons df
path_polygons

,efscale,geometry
0,EF3,"POLYGON ((-85.59922 36.17660, -85.59840 36.176..."
1,EF1,"POLYGON ((-85.52390 36.17189, -85.52430 36.171..."
2,EF1,"POLYGON ((-85.61527 36.17529, -85.61403 36.175..."
3,EF3,"POLYGON ((-85.55185 36.17398, -85.55184 36.173..."
4,EF0,"POLYGON ((-86.94788 36.17248, -86.94582 36.172..."
5,EF0,"POLYGON ((-84.79918 36.12665, -84.79661 36.124..."
6,EF2,"POLYGON ((-86.31818 36.84809, -86.31659 36.847..."
7,EF0,"POLYGON ((-87.62268 32.68464, -87.62261 32.684..."
8,EF0,"POLYGON ((-87.04970 32.86477, -87.03895 32.861..."
9,EF1,"POLYGON ((-87.04224 32.86482, -87.03977 32.863..."


In [29]:
#chekcing out tornado_paths df
tornado_paths

,objectid,event_id,stormdate,starttime,endtime,startlat,startlon,endlat,endlon,length,...,propdamage,edit_user,edit_time,created_user,created_date,last_edited_user,last_edited_date,comments,Shape__Length,geometry
0,879879,Nashville,1583217120000,1583217120000,1583220900000,36.1725,-86.9478,36.1532,-85.8860,60.1317,...,-99,None,None,DATians,1583461561000,dat_editor,1618703241000,"An historic, long-track, strong EF-3 tornado b...",1.071075,"LINESTRING (-86.94783 36.17249, -86.94461 36.1..."
1,879077,Hwy 69 Ballpark Tornado,1583237040000,1583237040000,1583237520000,32.6846,-87.6226,32.6863,-87.5492,4.3400,...,-99,None,None,DATians,1583353305000,DATians,1583358165000,None,0.074159,"LINESTRING (-87.62263 32.68464, -87.62182 32.6..."
2,879081,Lawley Tornado,1583238480000,1583238480000,1583239080000,32.8648,-87.0496,32.8646,-86.9432,6.2100,...,-99,None,None,DATians,1583357522000,DATians,1583359203000,Path,0.106695,"LINESTRING (-87.04956 32.86476, -87.04019 32.8..."
3,879877,Cookeville,1583221680000,1583221680000,1583222160000,36.1715,-85.6628,36.1709,-85.5135,8.3900,...,100000,None,None,DATians,1583461208000,DATians,1590771008000,"An historic, violent EF-4 tornado began in wes...",0.149870,"LINESTRING (-85.66285 36.17151, -85.61103 36.1..."
4,879878,Goffton,1583222700000,1583222700000,1583222760000,36.1075,-85.4461,36.1068,-85.4420,0.2300,...,-99,None,None,DATians,1583461360000,DATians,1590604073000,A very brief and weak EF-0 tornado touched dow...,0.004130,"LINESTRING (-85.44610 36.10746, -85.44203 36.1..."
5,879880,Cumberland,1583223900000,1583223900000,1583224500000,36.1399,-85.0401,36.1243,-84.8619,10.0700,...,-99,None,None,DATians,1583534817000,DATians,1590605084000,An EF-2 tornado touched down near the intersec...,0.179387,"LINESTRING (-85.04006 36.13989, -85.03400 36.1..."
6,888677,Buffalo Valley,1583221080000,1583221080000,1583221320000,36.1602,-85.8055,36.1711,-85.7479,3.3233,...,-99,None,None,DATians,1586271666000,DATians,1598799052000,An EF-0 tornado touched down near the Smith/Pu...,0.058826,"LINESTRING (-85.80550 36.16017, -85.79778 36.1..."


In [30]:
damage_points.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3967 entries, 0 to 3966
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   objectid            3967 non-null   int64   
 1   stormdate           3967 non-null   int64   
 2   surveydate          3967 non-null   int64   
 3   event_id            3967 non-null   object  
 4   damage              3967 non-null   int64   
 5   damage_txt          3967 non-null   object  
 6   dod_txt             3967 non-null   object  
 7   efscale             3967 non-null   object  
 8   damage_dir          3967 non-null   object  
 9   windspeed           3967 non-null   object  
 10  injuries            3967 non-null   int64   
 11  deaths              3967 non-null   int64   
 12  lat                 3967 non-null   float64 
 13  lon                 3967 non-null   float64 
 14  office              3967 non-null   object  
 15  image               3963 non-n

In [31]:
path_polygons.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   efscale   37 non-null     object  
 1   geometry  37 non-null     geometry
dtypes: geometry(1), object(1)
memory usage: 720.0+ bytes


In [32]:
tornado_paths.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   objectid          7 non-null      int64   
 1   event_id          7 non-null      object  
 2   stormdate         7 non-null      int64   
 3   starttime         7 non-null      int64   
 4   endtime           7 non-null      int64   
 5   startlat          7 non-null      float64 
 6   startlon          7 non-null      float64 
 7   endlat            7 non-null      float64 
 8   endlon            7 non-null      float64 
 9   length            7 non-null      float64 
 10  width             7 non-null      int64   
 11  injuries          7 non-null      int64   
 12  fatalities        7 non-null      int64   
 13  efscale           7 non-null      object  
 14  efnum             7 non-null      int64   
 15  qc                7 non-null      object  
 16  maxwind           7 no

In [33]:
##FINDING CENTERPOINT OF NASHVILLE SO I CAN CREATE A FOLIUM MAP OF NASH
nash_center = [36.174465, -86.767960]
print(nash_center)

[36.174465, -86.76796]


In [ ]:
#CREATING INITIAL FOLIUM MAP OF NASHVILLE
nashmap = folium.Map(location = nash_center, zoom_start = 12)


# Nashville Folium Map of the Tornado Path W/ Tornado CAD Incidents Overlay

In [ ]:
##CREATING FOLIUM MAP OF TORNADO PATH THROUGH NASHVILLE WITH TORNADO CAD INCIDENTS OVERLAY
nashmap = folium.Map(location = nash_center, zoom_start = 12)

folium.GeoJson(tornado_paths).add_to(nashmap)

for row_index, row_values in tornado_cad_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values['Location']), str(row_values['IncidentDate'])
    icon = folium.Icon(color = 'red', icon = 'car', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap)
nashmap

# Nashville Folium Map Of Torndao W/ 911 Calls Overlay.

In [ ]:
##CREATING FOLIUM MAP OF TORNADO PATH THROUGH NASHVILLE WITH TORNADO 911 CALLS OVERLAY.
nashmap1 = folium.Map(location = nash_center, zoom_start = 12)

folium.GeoJson(tornado_paths).add_to(nashmap1)

for row_index, row_values in tornado911_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'blue', icon = 'car', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap1)
nashmap1

# Nashville Folium Map of Tornado Path W/ Overlays of Both CAD Incidents & 911 Calls.

In [ ]:
###THIS IS MAP OF THE NASHVILLE TORNADO PATH WITH OVERLAYS OF BOTH TORNADO CAD INCIDENTS AND TORNADO 911 CALLS.
nashmap2 = folium.Map(location = nash_center, zoom_start = 12)

folium.GeoJson(tornado_paths).add_to(nashmap2)

for row_index, row_values in tornado_cad_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values['Location']), str(row_values['IncidentDate'])
    icon = folium.Icon(color = 'red', icon = 'car', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap2);

for row_index, row_values in tornado911_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'blue', icon = 'bus', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap2)    
nashmap2

# Nashville Folium Map of the Damage Paths

In [ ]:
nashmap3 = folium.Map(location = nash_center, zoom_start = 12)

folium.GeoJson(path_polygons).add_to(nashmap3)
nashmap3

# Nashville Folium Map of Damage Points

In [ ]:
nashmap4 = folium.Map(location = nash_center, zoom_start = 12)

folium.GeoJson(damage_points).add_to(nashmap4)
nashmap4

# Folium Map Combo of Damage Points and Path Polygons

In [ ]:
nashmap5 = folium.Map(location = nash_center, zoom_start = 12)

folium.GeoJson(damage_points).add_to(nashmap5)
folium.GeoJson(path_polygons).add_to(nashmap5)
nashmap5

# Folium Map of Damage Paths with 911 calls overlay

In [ ]:
nashmap6 = folium.Map(location = nash_center, zoom_start = 12)

folium.GeoJson(path_polygons).add_to(nashmap6)

for row_index, row_values in tornado911_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'blue', icon = 'bus', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap6)  
nashmap6

# Folium Map of Damage Paths with CAD Incident Overlay

In [ ]:
nashmap7 = folium.Map(location = nash_center, zoom_start = 12)

folium.GeoJson(path_polygons).add_to(nashmap7)

for row_index, row_values in tornado_cad_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values['Location']), str(row_values['IncidentDate'])
    icon = folium.Icon(color = 'red', icon = 'car', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap7);
nashmap7

# Folium Map of Nashville with Derecho 911 Calls & Cad Incidents

In [ ]:
nashmap8 = folium.Map(location = nash_center, zoom_start = 12)

for row_index, row_values in derecho_cad_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values['Location']), str(row_values['IncidentDate'])
    icon = folium.Icon(color = 'red', icon = 'car', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap8);
    
for row_index, row_values in derecho911_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'blue', icon = 'bus', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap8)  
    
nashmap8

### Creating a df of only electrical hazards (the #1 call) as result of the derecho.

In [ ]:
electrical = ['ELECTRICAL HAZARD']
deh = derecho_cad_geo.query("IncidentTypeDescription1 in @electrical")

In [ ]:
deh

# Folium Map of all Electrical Hazards during the derecho

In [ ]:
nashmap9 = folium.Map(location = nash_center, zoom_start = 12)

for row_index, row_values in deh.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values['Location']), str(row_values['IncidentDate'])
    icon = folium.Icon(color = 'red', icon = 'car', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap9);
nashmap9

In [ ]:
ind_alarm = ['GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD']
t_alarm = tornado_cad_geo.query("IncidentTypeDescription1 in @ind_alarm")
t_alarm

# Folium Map of Tornado Path with Industrial Buiding Alarm (#1 Call)
### Here is where I happend upon a rough timeline of the tornado path. If you click on one of the markers on the far left, you can see the tornado started roughly at 12:36am and then on the furthest right, it was on its way out of hermitage at approx. 12:56am  Most of the points along the tornado path follow that timeframe, a couple are from 0200 hours but if those are removed a general timeline will be established.

In [ ]:
nashmap10 = folium.Map(location = nash_center, zoom_start = 12)

folium.GeoJson(tornado_paths).add_to(nashmap10)

for row_index, row_values in t_alarm.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values['Location']), str(row_values['IncidentDate'])
    icon = folium.Icon(color = 'red', icon = 'car', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap10);
nashmap10

In [34]:
pd.to_datetime(tornado911['Seizure DateTime'])

0     2020-03-03 00:34:00
1     2020-03-03 00:36:00
2     2020-03-03 00:36:00
4     2020-03-03 00:36:00
5     2020-03-03 00:37:00
              ...        
470   2020-03-03 04:54:00
471   2020-03-03 04:54:00
472   2020-03-03 04:55:00
473   2020-03-03 04:56:00
474   2020-03-03 04:58:00
Name: Seizure DateTime, Length: 450, dtype: datetime64[ns]

In [35]:
tornado911['Seizure DateTime'] = pd.to_datetime(tornado911['Seizure DateTime'])
tornado911.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 0 to 474
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Seizure DateTime    450 non-null    datetime64[ns]
 1   CallTypeId          450 non-null    object        
 2   ALI Latitude        450 non-null    float64       
 3   ALI Longitude       450 non-null    float64       
 4   Cell Tower Address  450 non-null    object        
 5   geometry            450 non-null    geometry      
dtypes: datetime64[ns](1), float64(2), geometry(1), object(2)
memory usage: 24.6+ KB


# Pulling out the timeframe when the tornado 911 calls came in during 1st 5 minutes, 2nd 5 minutes, 3rd 5 minutes, and 4th 5 minutes

In [36]:
tornado911[((tornado911['Seizure DateTime'] >'2020-03-03 00:30:00') & 
            (tornado911['Seizure DateTime'] <= '2020-03-03 00:35:00'))]

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address,geometry
0,2020-03-03 00:34:00,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector,POINT (-86.66578 36.07271)


In [37]:
tornado911_first5 = tornado911[((tornado911['Seizure DateTime'] >'2020-03-03 00:35:00') & 
            (tornado911['Seizure DateTime'] <= '2020-03-03 00:40:00'))]
tornado911_second5 = tornado911[((tornado911['Seizure DateTime'] >'2020-03-03 00:40:00') & 
            (tornado911['Seizure DateTime'] <= '2020-03-03 00:45:00'))]
tornado911_third5 = tornado911[((tornado911['Seizure DateTime'] >'2020-03-03 00:45:00') & 
            (tornado911['Seizure DateTime'] <= '2020-03-03 00:50:00'))]
tornado911_last5 = tornado911[((tornado911['Seizure DateTime'] >'2020-03-03 00:50:00') & 
            (tornado911['Seizure DateTime'] <= '2020-03-03 01:00:00'))]
tornado911_after = tornado911[((tornado911['Seizure DateTime'] >'2020-03-03 01:00:00'))]

In [38]:
#Converting tornado911_west into a geodataframe
tornado911_first5_geo = gpd.GeoDataFrame(tornado911_first5, crs=('EPSG:4326'),geometry = tornado911_first5
                                       ['geometry'])
tornado911_second5_geo = gpd.GeoDataFrame(tornado911_second5, crs=('EPSG:4326'),geometry = 
                                          tornado911_second5['geometry'])
tornado911_third5_geo = gpd.GeoDataFrame(tornado911_third5, crs=('EPSG:4326'),geometry = tornado911_third5
                                       ['geometry'])
tornado911_last5_geo = gpd.GeoDataFrame(tornado911_last5, crs=('EPSG:4326'),geometry = tornado911_last5
                                       ['geometry'])
tornado911_after_geo = gpd.GeoDataFrame(tornado911_after, crs=('EPSG:4326'),geometry = tornado911_after
                                       ['geometry'])

# Plotting all 911 calls that came in during time intervals on folium map 

In [ ]:
nashmap11 = folium.Map(location = nash_center, zoom_start = 12)

folium.GeoJson(tornado_paths).add_to(nashmap11)

for row_index, row_values in tornado911_first5_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'blue', icon = 'phone', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap11);
    
for row_index, row_values in tornado911_second5_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'red', icon = 'phone', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap11);
    
for row_index, row_values in tornado911_third5_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'purple', icon = 'phone', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap11);
    
for row_index, row_values in tornado911_last5_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'black', icon = 'phone', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap11);
    
for row_index, row_values in tornado911_after_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values["ALI Longitude"]]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'orange', icon = 'phone', prefix = 'fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap11)
nashmap11

# CONVERTING TORNADO_CAD INCIDENT DATE COLUMN TO PD DATETIME

In [39]:
tornado_cad['IncidentDate'] = pd.to_datetime(tornado_cad['IncidentDate'])
tornado911.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 0 to 474
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Seizure DateTime    450 non-null    datetime64[ns]
 1   CallTypeId          450 non-null    object        
 2   ALI Latitude        450 non-null    float64       
 3   ALI Longitude       450 non-null    float64       
 4   Cell Tower Address  450 non-null    object        
 5   geometry            450 non-null    geometry      
dtypes: datetime64[ns](1), float64(2), geometry(1), object(2)
memory usage: 24.6+ KB


# SEPARATING TORNADO CAD INTO TIMEFRAMES

In [41]:
tornado_cad_first5 = tornado_cad[((tornado_cad['IncidentDate'] >'2020-03-03 00:35:00') & 
            (tornado_cad['IncidentDate'] <= '2020-03-03 00:40:00'))]
tornado_cad_second5 = tornado_cad[((tornado_cad['IncidentDate'] >'2020-03-03 00:40:00') & 
            (tornado_cad['IncidentDate'] <= '2020-03-03 00:45:00'))]
tornado_cad_third5 = tornado_cad[((tornado_cad['IncidentDate'] >'2020-03-03 00:45:00') & 
            (tornado_cad['IncidentDate'] <= '2020-03-03 00:50:00'))]
tornado_cad_last5 = tornado_cad[((tornado_cad['IncidentDate'] >'2020-03-03 00:50:00') & 
            (tornado_cad['IncidentDate'] <= '2020-03-03 01:00:00'))]
tornado_cad_after = tornado_cad[((tornado_cad['IncidentDate'] >'2020-03-03 01:00:00'))]

# CREATING GEODF from above.

In [42]:
tornado_cad_first5_geo = gpd.GeoDataFrame(tornado_cad_first5, crs=('EPSG:4326'),
                                          geometry = tornado_cad_first5['geometry'])
tornado_cad_second5_geo = gpd.GeoDataFrame(tornado_cad_second5, crs=('EPSG:4326'),
                                          geometry = tornado_cad_second5['geometry'])
tornado_cad_third5_geo = gpd.GeoDataFrame(tornado_cad_third5, crs=('EPSG:4326'),
                                          geometry = tornado_cad_third5['geometry'])
tornado_cad_last5_geo = gpd.GeoDataFrame(tornado_cad_last5, crs=('EPSG:4326'),
                                          geometry = tornado_cad_last5['geometry'])
tornado_cad_after_geo = gpd.GeoDataFrame(tornado_cad_after, crs=('EPSG:4326'),
                                          geometry = tornado_cad_after['geometry'])

In [ ]:
nashmap12 = folium.Map(location = nash_center, zoom_start = 12)

folium.GeoJson(tornado_paths).add_to(nashmap12)

for row_index, row_values in tornado_cad_first5_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values['IncidentDate'])
    icon = folium.Icon(color = 'blue', icon = 'warning', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap12);
    
for row_index, row_values in tornado_cad_second5_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values['IncidentDate'])
    icon = folium.Icon(color = 'red', icon = 'warning', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap12);
    
for row_index, row_values in tornado_cad_third5_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values['IncidentDate'])
    icon = folium.Icon(color = 'purple', icon = 'warning', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap12);
    
for row_index, row_values in tornado_cad_last5_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values['IncidentDate'])
    icon = folium.Icon(color = 'black', icon = 'warning', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap12);
    
for row_index, row_values in tornado_cad_after_geo.iterrows():
    loc = [row_values['Latitude1'], row_values["Longitude1"]]
    pop = str(row_values['IncidentDate'])
    icon = folium.Icon(color = 'orange', icon = 'warning', prefix = 'fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap12)
nashmap12

In [43]:
pd.to_datetime(derecho911['Seizure DateTime'])

0      2020-05-03 16:30:00
1      2020-05-03 16:30:00
2      2020-05-03 16:30:00
3      2020-05-03 16:30:00
4      2020-05-03 16:31:00
               ...        
1543   2020-05-04 00:16:00
1544   2020-05-04 00:18:00
1545   2020-05-04 00:22:00
1546   2020-05-04 00:25:00
1547   2020-05-04 00:29:00
Name: Seizure DateTime, Length: 1483, dtype: datetime64[ns]

In [44]:
derecho911['Seizure DateTime'] = pd.to_datetime(derecho911['Seizure DateTime'])
derecho911.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1483 entries, 0 to 1547
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Seizure DateTime    1483 non-null   datetime64[ns]
 1   CallTypeId          1483 non-null   object        
 2   ALI Latitude        1483 non-null   float64       
 3   ALI Longitude       1483 non-null   float64       
 4   Cell Tower Address  1483 non-null   object        
 5   geometry            1483 non-null   geometry      
dtypes: datetime64[ns](1), float64(2), geometry(1), object(2)
memory usage: 81.1+ KB


C:\Users\hbn2b\AppData\Local\Temp/ipykernel_16456/2874632106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derecho911['Seizure DateTime'] = pd.to_datetime(derecho911['Seizure DateTime'])


In [45]:
derecho911[((derecho911['Seizure DateTime'] >'2020-05-03 16:30:00') & 
            (derecho911['Seizure DateTime'] <= '2020-05-03 16:35:00'))]

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address,geometry
4,2020-05-03 16:31:00,911 Calls,36.142380,-86.881882,3744B ANNEX AVE - SE,POINT (-86.88188 36.14238)
5,2020-05-03 16:31:00,911 Calls,36.136683,-86.861322,3744B ANNEX AVE - SE,POINT (-86.86132 36.13668)
6,2020-05-03 16:31:00,911 Calls,36.047151,-86.962538,WIRELESS CALLER,POINT (-86.96254 36.04715)
7,2020-05-03 16:32:00,911 Calls,36.041454,-86.665907,WIRELESS CALLER,POINT (-86.66591 36.04145)
8,2020-05-03 16:32:00,911 Calls,36.093789,-86.839306,5043 G HILLSBORO PIKE - W,POINT (-86.83931 36.09379)
9,2020-05-03 16:32:00,911 Calls,36.030716,-86.981613,8621C HASELTON RD - NE,POINT (-86.98161 36.03072)
10,2020-05-03 16:32:00,911 Calls,36.111588,-86.888101,796 DAVIDSON DR - SE,POINT (-86.88810 36.11159)
11,2020-05-03 16:32:00,911 Calls,36.173944,-86.759934,WIRELESS CALLER,POINT (-86.75993 36.17394)
12,2020-05-03 16:33:00,911 Calls,36.113273,-86.842482,2205 CRESTMOOR RD - W,POINT (-86.84248 36.11327)
13,2020-05-03 16:33:00,911 Calls,36.097105,-86.838834,5043 G HILLSBORO PKE - W,POINT (-86.83883 36.09710)


In [54]:
derecho911_first10 = derecho911[((derecho911['Seizure DateTime'] >'2020-05-03 16:30:00') & 
            (derecho911['Seizure DateTime'] <= '2020-05-03 16:40:00'))]
derecho911_second10 = derecho911[((derecho911['Seizure DateTime'] >'2020-05-03 16:40:00') & 
            (derecho911['Seizure DateTime'] <= '2020-05-03 16:50:00'))]
derecho911_third10 = derecho911[((derecho911['Seizure DateTime'] >'2020-05-03 16:50:00') & 
            (derecho911['Seizure DateTime'] <= '2020-05-03 17:00:00'))]
derecho911_after = derecho911[((derecho911['Seizure DateTime'] >'2020-05-03 17:00:00'))]

In [55]:
derecho911_first10_geo = gpd.GeoDataFrame(derecho911_first10, crs=('EPSG:4326'),geometry = 
                                          derecho911_first10['geometry'])
derecho911_second10_geo = gpd.GeoDataFrame(derecho911_second10, crs=('EPSG:4326'),geometry = 
                                          derecho911_second10['geometry'])
derecho911_third10_geo = gpd.GeoDataFrame(derecho911_third10, crs=('EPSG:4326'),geometry = 
                                          derecho911_third10['geometry'])
derecho911_after_geo = gpd.GeoDataFrame(derecho911_after, crs=('EPSG:4326'),geometry = derecho911_after
                                       ['geometry'])

In [61]:
nashmap13 = folium.Map(location = nash_center, zoom_start = 12)

for row_index, row_values in derecho911_first10_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'blue', icon = 'phone', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap13);
    
for row_index, row_values in derecho911_second10_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'red', icon = 'phone', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap13);
    
for row_index, row_values in derecho911_third10_geo.iterrows():
    loc = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop = str(row_values['Seizure DateTime'])
    icon = folium.Icon(color = 'purple', icon = 'phone', prefix='fa')
    
    marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    marker.add_to(nashmap13);
    
#for row_index, row_values in derecho911_after_geo.iterrows():
    #loc = [row_values['ALI Latitude'], row_values["ALI Longitude"]]
    #pop = str(row_values['Seizure DateTime'])
    #icon = folium.Icon(color = 'orange', icon = 'phone', prefix = 'fa')
    
    #marker = folium.Marker(location = loc, popup = pop, icon = icon)
    
    #marker.add_to(nashmap13)
folium.LayerControl().add_to(nashmap13)
nashmap13